# Momentum Screener


The gist of this methodology is essentially to:

    1.) Penalize for Volatility
    2.) Buy non-news driven momentum - FIP helps filter for this


Process:

1.) Omit top % decile of the most volatile stocks in our universe

2.) 1yr/6month Volatiltiy Adjusted Returns: Calculate 1yr and 6month volatiltiy-adjusted returns and sort them highest to lowest - based on a combined 1yr & 6month score

3.) FIP (Momentum Quality): Calculate FIP for the top 50% of the stocks from the vol-adjsuted reteurn screen

4.) Rank Tickers: Calculate combined score based on the 1yr/6month vol-adjusted score and also the FIP score

5.) Purchase the top 40-50 equities from the universe (or a % based)

6.) Volatility Targetting: Volatility Targetted position sizing based on desired portfolio vol

7.) Rebalance: Rebalance monthly OR rebalance based on EV > Cost of rebalancing

In [224]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.linear_model import LinearRegression

In [226]:
sp500_tickers = pd.read_csv('sp500_tickers.csv')  # Assuming you have a file of S&P500 tickers
sp500_tickers.head()

,ticker
0,AAPL
1,MSFT
2,GOOG
3,AMZN
4,TSLA


# Omit Top Decile of Highest 1yr Volatility

In [229]:
# Create a blank dataframe with the appropriate columns
df = pd.DataFrame(columns=['Ticker', 'Historical Volatility'])

In [231]:
# Collect rows in a list for concatenation
rows_to_add = []

# Convert tickers column to a list
tickers = sp500_tickers['ticker'].tolist()

# Batch download all tickers at once
data = yf.download(tickers, period='2y', group_by='ticker', threads=True, auto_adjust=False)

for ticker in tickers:  # Iterate over the 'ticker' list
    try:
        # Retrieve individual ticker data from multi-ticker DataFrame
        ticker_data = data[ticker].copy()

        # Check if data is insufficient
        if ticker_data.empty or len(ticker_data) < 252:  
            raise ValueError(f"Insufficient data for ticker: {ticker}") 

        # Calculate daily returns
        ticker_data['Daily Return'] = ticker_data['Close'].pct_change()

        # Rolling window for std dev (daily returns)
        window = 252  

        # Calculate rolling standard deviation of daily returns
        ticker_data['Rolling_Std'] = ticker_data['Daily Return'].rolling(window).std()

        # Annualize rolling standard deviation to get historical annual volatility
        ticker_data['Rolling_Hist_Vol'] = ticker_data['Rolling_Std'] * np.sqrt(window)

        # Retrieve the last value of Vol Adjusted Return
        historical_vol = ticker_data['Rolling_Hist_Vol'].iat[-1]

        # Add the result to the list
        rows_to_add.append({'Ticker': ticker, 'Historical Volatility': historical_vol})

    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")

# Concatenate all rows into the DataFrame at once
df = pd.concat([pd.DataFrame(rows_to_add)], ignore_index=True)

[*********************100%***********************]  848 of 848 completed

9 Failed downloads:
['DFS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
['NARI', 'ENV', 'CTLT', 'MRO', 'TPX', 'PDCO', 'RCM', 'AZPN']: YFPricesMissingError('possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")')
C:\Users\eabiz\AppData\Local\Temp\ipykernel_6316\1293045128.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ticker_data['Daily Return'] = ticker_data['Close'].pct_change()
C:\Users\eabiz\AppData\Local\Temp\ipykernel_6316\1293045128.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_chang

In [233]:
df = df.sort_values('Historical Volatility', ascending=False) #highest Vol at the top
df.head()

,Ticker,Historical Volatility
516,WOLF,2.292434
441,LUMN,1.279683
753,SMCI,1.137417
363,SEDG,1.130233
685,RUN,1.063495


In [235]:
df = df.sort_values('Historical Volatility', ascending=False) #highest Vol at the top

# Determine the 90th percentile threshold (top 10% cutoff)
vol_threshold = df['Historical Volatility'].quantile(0.90)

# Drop tickers in the top decile
df = df[df['Historical Volatility'] <= vol_threshold]

df.head()

,Ticker,Historical Volatility
682,FLR,0.540534
210,ANET,0.540177
453,NRG,0.540112
627,PII,0.539353
147,CNC,0.538809


In [237]:
sp500_tickers = df # rename dataframe to sp500 tickers for the loops

sp500_tickers.drop(columns=['Historical Volatility'],inplace=True)
sp500_tickers.rename(columns={'Ticker' : 'ticker'},inplace=True)
sp500_tickers.head()

,ticker
682,FLR
210,ANET
453,NRG
627,PII
147,CNC


# Volatility Adjusted Returns

#### 1.) 1 Year Volatility-Adjusted Returns

In [241]:
# Create a blank dataframe with the appropriate columns
df = pd.DataFrame(columns=['Ticker', 'Vol Adjusted Return'])

In [243]:
# Collect rows in a list for concatenation
rows_to_add = []

# Convert tickers column to a list
tickers = sp500_tickers['ticker'].tolist()

# Batch download 2 years of data for all tickers at once
data = yf.download(tickers, period='2y', group_by='ticker', threads=True, auto_adjust=False)

for ticker in tickers:  # Iterate over the 'ticker' column in sp500_tickers dataframe
    try:
        # Retrieve individual ticker data from multi-ticker DataFrame
        ticker_data = data[ticker].copy()

        # Check if data is insufficient
        if ticker_data.empty or len(ticker_data) < 252:  
            raise ValueError(f"Insufficient data for ticker: {ticker}")

        # Calculate daily returns
        ticker_data['Daily Return'] = ticker_data['Close'].pct_change()

        # Calculate percent return over a 1-year period (252 trading days)
        ticker_data['Percent Return'] = ((ticker_data['Close'] - ticker_data['Close'].shift(252)) / ticker_data['Close'].shift(252)) * 100  

        # Rolling window for std dev (daily returns)
        window = 252  

        # Calculate rolling standard deviation of daily returns
        ticker_data['Rolling_Std'] = ticker_data['Daily Return'].rolling(window).std()

        # Annualize rolling standard deviation to get historical annual volatility
        ticker_data['Rolling_Hist_Vol'] = ticker_data['Rolling_Std'] * np.sqrt(window)

        # Calculate volatility-adjusted returns
        ticker_data['Vol_Adjusted_Return'] = ((ticker_data['Percent Return'])/100) / (ticker_data['Rolling_Hist_Vol'])

        # Retrieve the last value of Vol Adjusted Return
        vol_adjusted_return = ticker_data['Vol_Adjusted_Return'].iat[-1]

        # Add the result to the list
        rows_to_add.append({'Ticker': ticker, 'Vol Adjusted Return': vol_adjusted_return})

    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")

# Concatenate all rows into the DataFrame at once
df = pd.concat([pd.DataFrame(rows_to_add)], ignore_index=True)

[*********************100%***********************]  756 of 756 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_6316\2641018204.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ticker_data['Daily Return'] = ticker_data['Close'].pct_change()
C:\Users\eabiz\AppData\Local\Temp\ipykernel_6316\2641018204.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ticker_data['Daily Return'] = ticker_data['Close'].pct_change()
C:\Users\eabiz\AppData\Local\Temp\ipykernel_6316\2641018204.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a fut

In [245]:
df = df.sort_values('Vol Adjusted Return', ascending=False)
# Create a csv file with the results
df.to_csv("Momentum Screen Results.csv", index=False) # set index = to false, or else it'll create a column with index

In [247]:
# read the csv file - momentum screen results
df1 = pd.read_csv("Momentum Screen Results.csv")

df1['Index1'] = df1.index # create column of the index position
df1.head()

,Ticker,Vol Adjusted Return,Index1
0,TPR,3.974857,0
1,HWM,2.641294,1
2,NFLX,2.601855,2
3,IBKR,2.582223,3
4,JBL,2.521692,4


### 2.) 6 Month Volatility Adjusted Returns

In [250]:
# Create a blank dataframe with the appropriate columns
df = pd.DataFrame(columns=['Ticker', 'Vol Adjusted Return'])

In [252]:
# Collect rows in a list for concatenation
rows_to_add = []

# Convert tickers column to a list
tickers = sp500_tickers['ticker'].tolist()

# Batch download 1 year of data for all tickers at once
data = yf.download(tickers, period='1y', group_by='ticker', threads=True, auto_adjust=False)

for ticker in tickers:  # Iterate over the 'ticker' column in sp500_tickers dataframe
    try:
        # Retrieve individual ticker data from multi-ticker DataFrame
        ticker_data = data[ticker].copy()
        
        # Check if data is insufficient
        if ticker_data.empty or len(ticker_data) < 126:  
            raise ValueError(f"Insufficient data for ticker: {ticker}")

        # Calculate daily returns
        ticker_data['Daily Return'] = ticker_data['Close'].pct_change()

        # Calculate percent return over a 1-year period (252 trading days)
        ticker_data['Percent Return'] = ((ticker_data['Close'] - ticker_data['Close'].shift(126)) / ticker_data['Close'].shift(126)) * 100  

        # Rolling window for std dev (daily returns)
        window = 126  

        # Calculate rolling standard deviation of daily returns
        ticker_data['Rolling_Std'] = ticker_data['Daily Return'].rolling(window).std()

        # Annualize rolling standard deviation to get historical annual volatility
        ticker_data['Rolling_Hist_Vol'] = ticker_data['Rolling_Std'] * np.sqrt(252)

        # Calculate volatility-adjusted returns
        ticker_data['Vol_Adjusted_Return'] = ((ticker_data['Percent Return'])/100) / (ticker_data['Rolling_Hist_Vol'])

        # Retrieve the last value of Vol Adjusted Return
        vol_adjusted_return = ticker_data['Vol_Adjusted_Return'].iat[-1]

        # Add the result to the list
        rows_to_add.append({'Ticker': ticker, 'Vol Adjusted Return': vol_adjusted_return})

    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")

# Concatenate all rows into the DataFrame at once
df = pd.concat([pd.DataFrame(rows_to_add)], ignore_index=True)


[*********************100%***********************]  756 of 756 completed
C:\Users\eabiz\AppData\Local\Temp\ipykernel_6316\4286303003.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ticker_data['Daily Return'] = ticker_data['Close'].pct_change()
C:\Users\eabiz\AppData\Local\Temp\ipykernel_6316\4286303003.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ticker_data['Daily Return'] = ticker_data['Close'].pct_change()
C:\Users\eabiz\AppData\Local\Temp\ipykernel_6316\4286303003.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a fut

In [254]:
df = df.sort_values('Vol Adjusted Return', ascending=False)
# Create a csv file with the results
df.to_csv("6 Month Momentum Screen Results.csv", index=False) # set index = to false, or else it'll create a column with index

In [256]:
df2 = pd.read_csv("6 Month Momentum Screen Results.csv")
df2['Index2'] = df2.index # create column of the index position
df2.head()

,Ticker,Vol Adjusted Return,Index2
0,NEU,1.561230,0
1,ORA,1.556537,1
2,SXT,1.475955,2
3,LHX,1.413325,3
4,CW,1.298241,4


### Merge Both Dataframes - 1yr and 6M vol volatility adjusted returns

In [259]:
# Perform an inner join to include only matching tickers
df = pd.merge(df1, df2, on='Ticker', how='inner')
df.head()

,Ticker,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2
0,TPR,3.974857,0,0.872073,33
1,HWM,2.641294,1,1.093332,11
2,NFLX,2.601855,2,0.544475,91
3,IBKR,2.582223,3,0.309452,152
4,JBL,2.521692,4,0.804621,42


### Calculate Average Rank

In [262]:
# calculate average rank

df['average_rank'] = df['Index1'] + df['Index2'] # simply add the two ranks
df.head()

,Ticker,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2,average_rank
0,TPR,3.974857,0,0.872073,33,33
1,HWM,2.641294,1,1.093332,11,12
2,NFLX,2.601855,2,0.544475,91,93
3,IBKR,2.582223,3,0.309452,152,155
4,JBL,2.521692,4,0.804621,42,46


In [264]:
df = df.sort_values('average_rank', ascending=True)
# Create a csv file with the results
df.to_csv("Momentum Screen Results.csv", index=False) # set index = to false, or else it'll create a column with index
df.head()


###############

,Ticker,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2,average_rank
1,HWM,2.641294,1,1.093332,11,12
12,CW,2.174156,12,1.298241,4,16
8,WWD,2.302805,8,1.092466,12,20
9,NFG,2.266835,9,1.055326,13,22
18,APH,1.901053,18,1.259281,5,23


## Convexity of Returns

In [90]:
#only keep first 100 rows of the Momentum Screen Results 
df1 = df1.iloc[:100]

In [92]:
# Collect rows in a list for concatenation
rows_to_add_4 = []

# Initialize a results DataFrame
results_df = pd.DataFrame(columns=['Ticker', 'Convexity Ratio'])

for Ticker in df1['Ticker']:  # Iterate over the 'Ticker' column
    try:
        # Download 2 years of historical data
        data = yf.download(Ticker, period='2y')
        
        if data.empty or len(data) < 252:  # Check if data is insufficient
            raise ValueError(f"Insufficient data for ticker: {Ticker}")
        
        # Create a DataFrame for returns
        df = pd.DataFrame()
        df['Return'] = data['Adj Close'].pct_change()

        # Calculate the slope using a rolling window
        window = 50  # Adjust window size as needed
        df['Slope'] = df['Return'].rolling(window).apply(
            lambda x: np.polyfit(np.arange(len(x)), x, 1)[0], raw=True
        )

        # Compute the second derivative of the slope
        df['Second_Derivative'] = df['Slope'].diff()

        # Calculate the convexity ratio
        df['Convexity_Ratio'] = df['Second_Derivative'] / df['Slope']
        
        # Extract the most recent convexity ratio
        convexity_ratio = df['Convexity_Ratio'].iloc[-1]
        
        # Add the result to the list
        rows_to_add_4.append({'Ticker': Ticker, 'Convexity Ratio': convexity_ratio})
    
    except Exception as e:
        print(f"Error retrieving data for {Ticker}: {str(e)}")
        
# Concatenate all rows into the DataFrame at once
results_df = pd.concat([pd.DataFrame(rows_to_add_4)], ignore_index=True)

[*********************100%***********************]  1 of 1 completed


Error retrieving data for FOX: 'Adj Close'


[*********************100%***********************]  1 of 1 completed


Error retrieving data for FOXA: 'Adj Close'


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


Error retrieving data for COKE: 'Adj Close'


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Error retrieving data for PGR: 'Adj Close'
Error retrieving data for TTWO: 'Adj Close'


KeyboardInterrupt: 

In [56]:
df2 = results_df.sort_values(by='Convexity Ratio', ascending=False)

In [79]:
df2.head()

,Ticker,Convexity Ratio
46,MRK,7.440757
48,INTC,3.696518
44,PEP,2.642901
24,TXN,1.623321
20,XOM,1.375793


In [83]:
#Create a new csv file with our FIP results
df2.to_csv('Convexity_Results.csv', index=False)

# Convexity Based on Quadratic Regression

In [60]:
df = df.iloc[:400]

In [62]:
rows_to_add_2 = []

# Initialize an empty DataFrame to store results
df_convexity = pd.DataFrame(columns=['Ticker', 'Convexity Score'])

# Get tickers from df['Ticker'] list
tickers = df['Ticker'].tolist()

# Batch download 2 years of data for all tickers at once
data = yf.download(tickers, period='2y', group_by='ticker', threads=True, auto_adjust=False)

for ticker in tickers:
    try:
        # Retrieve individual ticker data from multi-ticker DataFrame
        ticker_data = data[ticker].copy()

        # Keep only last 252 rows... 252 trading days
        ticker_data = ticker_data.tail(252)

        # Ensure there is sufficient data
        if ticker_data.empty or len(ticker_data) < 252:  
            raise ValueError(f"Insufficient data for ticker: {ticker}")
        
        # Calculate daily returns (percentage change)
        ticker_data['Return'] = ticker_data['Close'].pct_change() * 100

        # Drop the first row since the return for the first day is NaN
        ticker_data.dropna(inplace=True)
        
        # Create an ordinal time variable (index of the data)
        ticker_data.loc[:, 'Time'] = np.arange(len(ticker_data))
        
        # Prepare the independent variables (Time and Time^2 for quadratic regression)
        X = np.vstack([ticker_data['Time'], ticker_data['Time']**2]).T
        y = ticker_data['Return'].values
        
        # Perform a quadratic regression
        model = LinearRegression().fit(X, y)
        
        # The convexity is the coefficient of the squared time term
        convexity = model.coef_[1]  # This is the coefficient of Time^2 (quadratic term)
        
        # Add the result to the list
        rows_to_add_2.append({'Ticker': ticker, 'Convexity Score': convexity})
    
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {str(e)}")

# Concatenate all rows into the DataFrame at once
df_convexity = pd.concat([pd.DataFrame(rows_to_add_2)], ignore_index=True)


[*********************100%***********************]  14 of 14 completed


In [64]:
df_convexity = df_convexity.sort_values('Convexity Score', ascending=False)

In [66]:
df_convexity.reset_index(drop=True, inplace=True)

In [68]:
df_convexity.head()

,Ticker,Convexity Score
0,OKE,0.000020
1,NI,0.000013
2,TATT,0.000007
3,TPR,0.000007
4,CLBT,0.000006


In [70]:
df_convexity.to_csv("Convexity_Score_Quadratic_Method.csv")


###

# Momentum Quality - FIP (Frog in the Pan Score)

ID or FIP Score
ID = sign x [% Negative - % Positive]

In [266]:
# Only keep first 400 rows of the Momentum Screen Results
df = df.iloc[:400]

In [268]:
# Create a blank dataframe with the appropriate columns
df_FIP = pd.DataFrame(columns=['Ticker', 'FIP_Score'])

In [270]:
rows_to_add_3 = []

# Get list of tickers from df
tickers = df['Ticker'].tolist()

# Batch download 2 years of data for all tickers at once
data = yf.download(tickers, period='2y', group_by='ticker', threads=True, auto_adjust=False)

for Ticker in tickers:  # Iterate over the 'ticker' column in df
    try:
        # Retrieve individual ticker data from multi-ticker DataFrame
        ticker_data = data[Ticker].copy()
        ticker_data = ticker_data.tail(252)  # Use only last 252 rows... for 252 trading days

        if ticker_data.empty or len(ticker_data) < 252:  # Check if data is insufficient
            raise ValueError(f"Insufficient data for ticker: {Ticker}")
        
        ticker_data['Percent Return'] = (ticker_data['Close'].pct_change()) * 100

        if ticker_data['Percent Return'].cumsum().iat[-1] > 0:
            sign = 1
        elif ticker_data['Percent Return'].cumsum().iat[-1] < 0:
            sign = -1
        else:
            sign = 0

        positive_days = (ticker_data['Percent Return'] > 0).sum()
        negative_days = (ticker_data['Percent Return'] < 0).sum()
        flat_days = (ticker_data['Percent Return'] == 0).sum()
        pct_positive = round(positive_days / len(ticker_data), 4)
        pct_negative = round(negative_days / len(ticker_data), 4)
        FIP_Score = round(sign * (pct_negative - pct_positive), 4)

        # Add the result to the list
        rows_to_add_3.append({'Ticker': Ticker, 'FIP Score': FIP_Score})

    except Exception as e:
        print(f"Error retrieving data for {Ticker}: {str(e)}")

df_FIP = pd.concat([pd.DataFrame(rows_to_add_3)], ignore_index=True)


[*********************100%***********************]  394 of 394 completed


In [272]:
df_FIP = df_FIP.sort_values('FIP Score', ascending=True) # ascending = True since the best scores are negative

In [274]:
df_FIP.reset_index(drop=True, inplace=True)

In [276]:
df_FIP.head()

,Ticker,FIP Score
0,DTE,-0.2381
1,CW,-0.2143
2,NDAQ,-0.2143
3,GL,-0.2024
4,CME,-0.2024


In [278]:
# Create a new csv file with our FIP results
df_FIP.to_csv('Momentum_FIP_Results.csv', index=False)


###

# Combine Dataframes and Rank Tickers

In [280]:
#Create column of index #
df['index_vol_rank'] = df.index #vol adjusted returns df

#Create column of index #
df_FIP['index_fip_rank'] = df_FIP.index # FIP df

# create column of index # 
#df_convexity['Index3'] = df_convexity.index

In [282]:
# Perform an inner join to include only matching tickers
merged = pd.merge(df_FIP, df, on='Ticker', how='inner')

In [284]:
merged.head()

,Ticker,FIP Score,index_fip_rank,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2,average_rank,index_vol_rank
0,DTE,-0.2381,0,0.815923,140,0.844911,35,175,140
1,CW,-0.2143,1,2.174156,12,1.298241,4,16,12
2,NDAQ,-0.2143,2,1.609173,43,0.555771,89,132,43
3,GL,-0.2024,3,1.630861,42,0.382556,132,174,42
4,CME,-0.2024,4,2.321029,7,0.953891,24,31,7


### Next cell is only necessary if adding a third filter to the ranking:

In [286]:
# Merge the result with the third DataFrame
merged = pd.merge(merged, df_convexity, on='Ticker', how='inner')

In [288]:
merged.head()

,Ticker,FIP Score,index_fip_rank,Vol Adjusted Return_x,Index1,Vol Adjusted Return_y,Index2,average_rank,index_vol_rank
0,DTE,-0.2381,0,0.815923,140,0.844911,35,175,140
1,CW,-0.2143,1,2.174156,12,1.298241,4,16,12
2,NDAQ,-0.2143,2,1.609173,43,0.555771,89,132,43
3,GL,-0.2024,3,1.630861,42,0.382556,132,174,42
4,CME,-0.2024,4,2.321029,7,0.953891,24,31,7


In [290]:
# Calculate the score (sum of indices)

#merged['Score'] = merged['index_vol_rank'] + merged['index_fip_rank'] + merged['Index3']
merged['Score'] = merged['index_vol_rank'] + merged['index_fip_rank']

# Sort the result by the score
sorted_df = merged.sort_values('Score', ascending=True)
# Select relevant columns for the output
result = sorted_df[['Ticker', 'Score']]

In [292]:
result.head()

,Ticker,Score
4,CME,11
9,IBKR,12
1,CW,13
15,HWM,16
12,CAH,17


In [294]:
result.to_csv("Momentum_Ranked.csv",index=False)






#####